## LQTMOMENT Tutorial 2: Preparing Wave Data, Instrument Response File, and Configuration Parameters

Setting wave/seismogram data, instrument response file, and configuration parameters are crucial step in calculating moment magnitude using the `lqtmoment` package.


> **ℹ️ INFO ℹ️**
>
> In this current version, `lqtmoment` only accepts `.mseed` format for seismogram data and `RESP` files format for instrument response file. These two formats have become the standard for most organizations nowadays, and maintaining them for processing is considered good practice.


### A. Setting up The Seismogram Directory Structure

Suppose that under the `wave` directory, the seismogram data within this directory must be stored according to the following structure: each seismogram dataset is placed in a folder named with the corresponding earthquake ID.


```
wave/
├── 1001/
│   ├── 19700101T000000_KJ.KJ01_BHE.mseed
│   ├── 19700101T000000_KJ.KJ01_BHN.mseed
│   ├── 19700101T000000_KJ.KJ01_BHZ.mseed
│   ├── 19700101T000000_KJ.KJ02_BHE.mseed
│   ├── 19700101T000000_KJ.KJ02_BHN.mseed
│   ├── 19700101T000000_KJ.KJ02_BHZ.mseed
│   ├── 19700101T000000_KJ.KJ03_BHE.mseed
│   ├── 19700101T000000_KJ.KJ03_BHN.mseed
│   └── 19700101T000000_KJ.KJ03_BHZ.mseed
├── 1002/
│   ├── 19700101T010000_KJ.KJ01_BHE.mseed
│   ├── 19700101T010000_KJ.KJ01_BHN.mseed
│   ├── 19700101T010000_KJ.KJ01_BHZ.mseed
│   ├── 19700101T010000_KJ.KJ02_BHE.mseed
│   ├── 19700101T010000_KJ.KJ02_BHN.mseed
│   ├── 19700101T010000_KJ.KJ02_BHZ.mseed
│   ├── 19700101T010000_KJ.KJ03_BHE.mseed
│   ├── 19700101T010000_KJ.KJ03_BHN.mseed
│   └── 19700101T010000_KJ.KJ03_BHZ.mseed
├── 1003/
│   ├── 19700101T020000_KJ.KJ01_BHE.mseed
│   ├── 19700101T020000_KJ.KJ01_BHN.mseed
│   ├── 19700101T020000_KJ.KJ01_BHZ.mseed
│   ├── 19700101T020000_KJ.KJ02_BHE.mseed
│   ├── 19700101T020000_KJ.KJ02_BHN.mseed
│   ├── 19700101T020000_KJ.KJ02_BHZ.mseed
│   ├── 19700101T020000_KJ.KJ03_BHE.mseed
│   ├── 19700101T020000_KJ.KJ03_BHN.mseed
│   └── 19700101T020000_KJ.KJ03_BHZ.mseed
├── 1004/
│   ├── 19700101T030000_KJ.KJ01_BHE.mseed
│   ├── 19700101T030000_KJ.KJ01_BHN.mseed
│   ├── 19700101T030000_KJ.KJ01_BHZ.mseed
│   ├── 19700101T030000_KJ.KJ02_BHE.mseed
│   ├── 19700101T030000_KJ.KJ02_BHN.mseed
│   ├── 19700101T030000_KJ.KJ02_BHZ.mseed
│   ├── 19700101T030000_KJ.KJ03_BHE.mseed
│   ├── 19700101T030000_KJ.KJ03_BHN.mseed
│   └── 19700101T030000_KJ.KJ03_BHZ.mseed
├── 1005/
│   ├── 19700101T040000_KJ.KJ01_BHE.mseed
│   ├── 19700101T040000_KJ.KJ01_BHN.mseed
│   ├── 19700101T040000_KJ.KJ01_BHZ.mseed
│   ├── 19700101T040000_KJ.KJ02_BHE.mseed
│   ├── 19700101T040000_KJ.KJ02_BHN.mseed
│   ├── 19700101T040000_KJ.KJ02_BHZ.mseed
│   ├── 19700101T040000_KJ.KJ03_BHE.mseed
│   ├── 19700101T040000_KJ.KJ03_BHN.mseed
│   └── 19700101T040000_KJ.KJ03_BHZ.mseed
└── ...

> **⚠️ CAUTION ⚠️**
>
> Ideally, the naming convention for seismogram data in the `lqtmoment` package follows the format `<yyyymmddTHHMMSS>_<network_code>.<station_code>_<channel>.mseed`. However, `lqtmoment` is flexible with naming conventions and primarily relies on the `<station_code>` to identify the corresponding seismogram data. Adhering to the full naming convention for seismogram data is always considered best practice.
> 
>
> The length, or duration, of the wave data for each component of the seismogram is user-defined; make sure it captures the entire earthquake coda. Longer wave data durations will result in heavier loading times, while shorter durations will reduce loading times and vice versa.
>
> Tight trimming and rigorous downsampling can indeed improve loading times, but be cautious of frequency leakage and aliasing. Allow some space for each tip; the tapering will be locally handled by `lqtmoment`.



### B. Preparing Instrument Response File

A digital seismogram typically stores its raw measurements in **count** units. To calculate moment magnitude, which is a physical property, we need to convert these digital units into **true ground displacement in meters (m)**. This is achieved through deconvolution, where the spectrum in digital units is divided by the instrument response function. To perform this accurately, the correct **instrument response file** is required.


**1. Getting Instrument Response File (RESP)**

Typically, your instrument vendor or provider will supply the correct datalogger and sensor response file combined in `RESP` format with every purchase. If these are not included or you get the data from open-source or third-party providers, you can retrieve the correct files through the [Iris Data Web Service](http://service.iris.edu/irisws/nrl/1/) by inputting the appropriate datalogger and sensor specifications.

For example:

* **Datalogger Specs**: REFTEK 130-01; Gain 32; 500 sps
* **Sensor Specs**: CEADASE CPHM-500; Long Period 1; Gain 8; Sensitivity 2250 V/m/s; Sensor Type Ground Vel

The web url query would be like this:

http://service.iris.edu/irisws/nrl/1/combine?instconfig=sensor_CEADASE_CPHM-500_LP1_PG8_SG2250_STgroundVel:datalogger_REFTEK_130-01_PG32_FR500&format=resp&nodata=404 

> **⚠️ CAUTION ⚠️**
>
> After obtaining the `RESP` file, you must still need to configure and align the actual seismogram metadata with the metadata form the `RESP` file you just donwloaded.
>
> Any discrepancy between the RESP metadata and the seismogram metadata will prevent lqtmoment from processing the waveform data, as it cannot parse the poles, zeros, and sensitivity constant from the RESP file.


**2. RESP file format**

A `RESP` file is an ASCII representation of SEED instrument response information. Generally RESP files only contain the subset of SEED headers that represent an instrument response. For more detail please check this out [The RESP format](http://ds.iris.edu/ds/nodes/dmc/data/formats/resp/#:~:text=A%20RESP%20file%20is%20an,metadata%20available%20in%20SEED%20headers.).

A `RESP` file generally contains these informations:

* **Station and Channel Info**: Network, Station, Location, Channel Codes, and Validity dates.
* **Poles and Zeros**: Defines the sensor's frequency response for deconvolution.
* **Sensitivity/Gain**: Scales raw counts to physical units (e.g., counts/m/s)
* **Sampling Rate**: Specifies the digitization frequency (e.g., 20 Hz)
* **Stage Sequence**: Describes the signal path (sensor, digitizer, filters) with units and gains.gains.



**3. Configuring the Directory Structure and Naming Conventions for RESP files**

The naming convention for `RESP` file is following this format.

`RESP.<network_code>.<station_code>.<location>.<channel>.resp`

for example:

`RESP.IU.FURI.00.BHE.resp`

> **⚠️ CAUTION ⚠️**
>
> The naming structure must correspond to the data header in the `RESP` file.

Data header in `RESP` usually contains these informations:

```
# ============= CHANNEL RESPONSE DATA ===============
B050F03        Station:        FURI
B050F16        Network:        IU
B052F03        Location:       00
B052F04        Channel:        BHE
B052F22        Start Date:     1999,111,00:00:00
B052F23        End Date: No Ending Time
```

> **⚠️ CAUTION ⚠️**
>
> Any discrepancy between the `RESP` header metadata and the naming convention will prevent `lqtmoment` from processing the waveform data, as it cannot parse the poles, zeros, and sensitivity constant from the RESP file.



So suppose under `calibration` folder, your directory structure should be looking like this:

```
wave/
├── RESP.IU.FURI.00.BHE.resp
├── RESP.IU.FURI.00.BHN.resp
├── RESP.IU.FURI.00.BHZ.resp
├── RESP.ZQ.AAMI.00.BHE.resp
├── RESP.ZQ.AAMI.00.BHN.resp
├── RESP.ZQ.AAMI.00.BHZ.resp
└── 

### C. Setting Up Configuration Parameters

When using `lqtmoment`, there are only two configuration files you need to prepare: the `config.ini`, which contains all the parameters necessary for computation, and `velocity_model.json` which exclusively stores the velocity model data. You can change the names of theses files but please keep the extensions to be always `.ini` and `.json`.



#### 1. The Configuration File (.ini)

The `config.ini` file contains important parameters required to run `lqtmoment`, so it's essential to configure it correctly. The details of the `config.ini` file are as follows (note that the parameters shown are just examples):
```ini
    [Wave]
    resample_data = None
    snr_threshold = 1.25
    pre_filter = 0.1,0.5,100,125
    water_level = 20.0
    apply_post_instrument_removal_filter = True
    post_filter_f_min = 3
    post_filter_f_max = 60
    trim_mode = dynamic
    sec_bf_p_arr_trim = 5
    sec_af_p_arr_trim = 25
    padding_before_arrival = 0.1
    min_p_window = 1.0
    max_p_window = 10.0
    min_s_window = 2.0
    max_s_window = 20.0
    noise_duration = 0.5
    noise_padding = 0.2

    [Magnitude]
    r_pattern_p = 0.52
    r_pattern_s = 0.63
    free_surface_factor = 2.0
    k_p = 0.32
    k_s = 0.21
    mw_constant = 6.07
    taup_model = iasp91
    velocity_model_file = velocity_model.json

    [Spectral]
    smooth_window_size = 3
    f_min = 1
    f_max = 50
    omega_0_range_min = 0.001
    omega_0_range_max = 1000
    q_range_min = 50
    q_range_max = 250
    default_n_samples = 3000
    n_factor = 2
    y_factor = 1

    [Performance]
    use_parallel = False
    logging_level = INFO
```

As you can see, there are four essential sections in the parameter settings. First, you need to set the parameters related to your waveform/seismogram data. Next are the parameters for moment magnitude calculation, followed by the spectral fitting parameters, and finally, the performance-related parameters.

> **⚠️ CAUTION ⚠️**
> 
> You need to follow and fill in all of these parameter fields; otherwise, the program will fall back to the default parameters in `lqtmoment`, which are likely not suitable for your specific case.
>


**The detailed explanation for each of the parameters mentioned above is as follows:**

##### A. Wave Part

This `Wave` section handles all the necessary settings for seismogram data processing, including resampling, instrument removal parameters, trimming, and more..

Detailed Parameters:

- **resample_data:** New sampling rate value to be applied to seismogram data (default: None)
- **snr_threshold:** Minimum signal-to-noise ratio for trace acceptance (unitless, default: 2.0)
- **pre_filter:** Pre-processing bandpass filter corners (f1,f2,f3,f4 in Hz, default: 0.001,0.005,55,60)
- **water_level:** Water level for deconvolution stabilization (unitless, default: 60)
- **apply_post_instrument_removal_filter:** If True, post filter after instrument removal will be applied (default: True).
- **post_filter_f_min:** Post-processing filter minimum frequency (Hz, default: 0.01)
- **post_filter_f_max:** Post-processing filter maximum frequency (Hz, default: 30.0)
- **trim_mede:** Mode used to trim, 'dynamic' or 'static', if 'dynamic' consider the coda_time in catalog.
- **sec_bf_p_arr_trim:** seconds before P arrival trim to start the trimming.
- **sec_af_p_arr_trim:** seconds after P arrival trim to end the trimming.
- **padding_before_arrival:** Padding before P/S arrival for signal window (seconds, default: 0.2).
- **min_p_window:** Minimum P phase window in second for calculating source spectra (default: 1.0).
- **max_p_window:** Maximum P phase window in second for calculating source spectra (default: 10.0).
- **min_s_window:** Minimum S phase window in second for calculating source spectra (default: 2.0).
- **max_s_window:** Maximum S phase window in second for calculating source spectra (default: 20.0).
- **noise_duration:** Duration of noise window (seconds, default: 1.0)
- **noise_padding:** Padding around noise window (seconds, default: 0.2)

> **⚠️ CAUTION ⚠️**
>
> If you use the `resample_data` feature, make sure your `.RESP` file matches the new `sampling rate`; otherwise, the response removal process to obtain true ground displacement will fail.
> 
> If you rely more on `pre-filter` before performing instrument response removal, you can simply set the `water-level` to `None`. This means that no numerical stabilization mechanism will be applied when dividing the raw seismogram data by the instrument response in the frequency domain. If you'd like to learn more about what the `water-level` parameter actually does, you can visit this resource [water-level](http://eqseis.geosc.psu.edu/cammon/HTML/RftnDocs/seq01.html).
>
> The parameters `apply_post_instrument_removal_filter`, `post_filter_f_min`, and `post_filter_f_max` should be considered if you want to apply post-instrument removal filtering. This means you're applying a filter to seismogram data that has already been converted to true ground displacement. This approach is beneficial because applying a `pre-filter` requires extra caution to avoid introducing instability during the deconvolution process—so you typically don't want to set it too strictly. By applying a post-filter afterward, you can enhance the signal-to-noise ratio (SNR) while also gaining more control over the final frequency range that matters for your analysis. If you don't want to do this, just set the `apply_post_instrument_removal_filter` to `false` and the`post_filter_f_min`, and `post_filter_f_max` will be neglected. 
>
> In `lqtmoment` a trimming procedure is applied to shorten your raw seismogram data to improve computation time so that it only includes the portion necessary for calculating moment magnitude. If you set the trimming mode to `dynamic`, program will use the `coda_time` provided in your catalog. If no `coda_time` is available, it will automatically fall back to the static value defined in `sec_af_p_arr_trim`. On the other hand, if you set `trim_mode` to `static`, the `coda_time` will be ignored even if it exists in your lqt catalog.
>
> Phases windowing is internal dynamic process within `lqtmoment`. The program ensures there is no phase contamination and minimizes the inclusion of unwanted signals in the calculation of source spectra. Users generally set the parameters `min_p_window`, `max_p_window`, `min_s_window`, and `max_s_window` according to their specific cases. Internally, `lqtmoment` aggregates these parameters with the `s_p_lag_time` setting to enable more robust windowing.



##### B. Magnitude Part

This `Magnitude` section handles all the parameters related to the equations used in calculating moment magnitude.

Detailed Parameters:

- **r_pattern_p:** Radiation pattern correction for P-waves (unitless, default: 0.52, Aki & Richards, 2002)
- **r_pattern_s:** Radiation pattern correction for S-waves (unitless, default: 0.63, Aki & Richards, 2002)
- **free_surface_factor:** Free surface amplification factor (unitless, default: 2.0)
- **k_p:** Geometric spreading factor for P-waves (unitless, default: 0.32)
- **k_s:** Geometric spreading factor for S-waves (unitless, default: 0.21)
- **mw_constant:** The empirical constant value for moment magnitude calculation
- **taup_model:** 1-D velocity model for estimating incident angle and source distance for regional to teleseismic earthquake (default: 'iasp91')
- **velocity_model_file:** Path to a JSON file defining the velocity model(default: None, uses built-in model)

> **⚠️ CAUTION ⚠️**
> 
> In the `velocity_model_file` parameter, you specify the path to the `.json` file containing your 1-D velocity model. You can use either an absolute or relative path (absolute path is more preferable). If this file is not specified, the velocity parameters will fall back to the default values used by the `lqtmoment` program, which may not be suitable for your specific case.
>


##### C. Spectral Fitting Part
This `Spectral` section handles all parameters necessary to perform spectral fitting.

Detailed Parameters:
- **smooth_window_size:** Size of the moving average window for smoothing, must be odd positive, if None no smoothing applied (default: 3).
- **f_min:** Minimum frequency for spectral fitting (Hz, default: 0.01)
- **f_max:** Maximum frequency for spectral fitting (Hz, default: 30)
- **omega_0_range_min:** Minimum Omega_0 for fitting (nm/Hz, default: 0.01) [Confirm units]
- **omega_0_range_max:** Maximum Omega_0 for fitting (mm/Hz, default: 2000) [Confirm units]
- **q_range_min:** Minimum Q factor for attenuation (unitless, default: 50)
- **q_range_max:** Maximum Q factor for attenuation (unitless, default: 300)
- **default_n_samples:** Number of Monte Carlo samples for fitting (default: 3000)
- **n_factor:** Stress drop exponent in Brune model (default: 2, Brune, 1970)
- **y_factor:** High-frequency fall-off exponent (default: 1, Brune; 2 for Boatwright)

> **⚠️ CAUTION ⚠️**
> 
> A higher value for `default_n_samples` will result in increased computational time.
>

##### D. Computation Performance Part
And the last, `Performance` section handles `lqtmoment` computation performance .

Detailed Parameters:

- **use_parallel:** Enable parallel processing (default: false)
- **logging_level:** Logging verbosity (DEBUG, INFO, WARNING, ERROR, default: INFO)

> **⚠️ CAUTION ⚠️**
> 
> In this version, `lqtmoment` does not support parallel computing. This feature is planned to be implemented in a future release once the program reaches a stable stage.

#### 2. The Velocity Model File (.json)

Setting the velocity model directly inside `config.ini` can be **difficult** and **error-prone**, which is why the `.json` file was created to make it easier for you to manage your velocity model data. This `.json` file should contain detailed information about `layer_boundaries`, `velocity_vp`, `velocity_vs`, and `density` data specific to your case.

The `layer_boundaries` parameter is a nested list that stores your top and bottom boundaries in the following format: `[[top_1, bottom_1], [top_2, bottom_2], and so on...]`. The negative `(-)` notation indicates boundaries above sea level or for elevation.

And `velocity_vp`, `velocity_vs`, and `density` are simple lists that store the P-wave velocities, S-wave velocities, and density values for each corresponding layer.

And here is an example of what the `.json` file should look like:

```json
    {
        "layer_boundaries": [ [-3.00,-1.90],[-1.90,-0.59],[-0.59, 0.22],[0.22, 2.50], [2.50, 7.00], [7.00,9.00] , [9.00,15.00], [15.00,33.00], [33.00,9999]],
        "velocity_vp": [2.68, 2.99, 3.95, 4.50, 4.99, 5.60, 5.80, 6.40, 8.00],
        "velocity_vs": [1.60, 1.79, 2.37, 2.69, 2.99, 3.35, 3.47, 3.83, 4.79],
        "density": [ 2375, 2375, 2375, 2465, 2529, 2750, 2750, 2931, 2931]
    }
```

Your final directory structure may look like this; however, you are free to organize it as you prefer. As long as all directory paths are correctly set when running `lqtmoment`, there should be no issues. It is also recommended to use absolute paths for all directories to prevent unexpected behavior.

``` ascii
working_directory
├── config
│   ├── config_test.ini
│   └── velocity_model_test.json
├── data
│   ├── calibration
│   ├── catalog
│   ├── station
│   └── wave
├── figures
└── results
    ├── calculation
    └── lqt_catalog
```